In [ ]:
%pip install fabric-analytics-sdk[online-notebook]

In [2]:
from fabric.analytics.environment.context import FabricContext
from fabric.analytics.environment.credentials import FabricAnalyticsTokenCredentials
from azure.core.paging import ItemPaged
from azure.storage.filedatalake import DataLakeServiceClient, PathProperties
from azure.storage.filedatalake._list_paths_helper import PathPropertiesPaged
from itertools import islice
context = FabricContext()
client = DataLakeServiceClient(account_url=context.onelake_endpoint, credential=FabricAnalyticsTokenCredentials()).get_file_system_client(context.workspace_id)

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 9, Finished, Available, Finished)

In [3]:
import functools

def get_paths(path=None, recursive=True, max_results=None, **kwargs):
        timeout = kwargs.pop("timeout", None)
        command = functools.partial(
            client._client.file_system.list_paths,
            path=path,
            timeout=timeout,
            **kwargs,
        )
        return ItemPaged(
            command,
            recursive,
            path=path,
            max_results=max_results,
            page_iterator_class=PathPropertiesPaged,
        )

def list_artifacts(path=None, recursive=True):
    if not path:
        print("hello")
        path = ""
    files: ItemPaged[PathProperties] = get_paths(
        path=path, recursive=recursive
    )
    res = []
    for f in files:
        res.append(
            {"path":f.name, "file_size":f.content_length, "is_dir": f.is_directory}
        )
    return res

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 10, Finished, Available, Finished)

In [4]:
def list_artifacts(path=None, recursive=True):
    if not path:
        path = ""
    files: ItemPaged[PathProperties] = get_paths(
        path=path, recursive=recursive
    )

    return files

    res = []
    for f in files:
        res.append(
            {"path":f.name, "file_size":f.content_length, "is_dir": f.is_directory}
        )
    return res

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 11, Finished, Available, Finished)

In [5]:
#ML Experiement artififact ID
paths = get_paths(path="xxx-xxx-xxx-xxx",recursive=True) # XGBoost_Yearly_Pred -> 0.34TB 152161 files

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 12, Finished, Available, Finished)

In [7]:

limit = 10**4
res = []
total_file_size = 0
total_file_count = 0
last_file_name = ''
last_file_size = 0

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 14, Finished, Available, Finished)

In [ ]:
while True:
    file_size = 0
    file_count = 0
    #First iterate trough limit amount of folders and files
    for f in islice(paths, limit):
        file_size += f.content_length
        file_count += 1

    #Read one more single file to get an exmple of a file
    for f in islice(paths, 1):
        file_size += f.content_length
        file_count += 1
        last_file_name = f.name
        last_file_size = f.content_length 

    total_file_size += file_size
    total_file_count += file_count

    if file_size == 0: 
        break

    print("Total File Size: ",total_file_size)
    print("Total File Count: ", total_file_count)
    print("Last File Name: ", last_file_name)
    print("Last File Size", last_file_size)

In [ ]:
print("Total File Size in TB: ",total_file_size / 1024**4)